# Transfer learning with Huggingface using CodeFlare

In this notebook you will learn how to leverage the **[huggingface](https://huggingface.co/)** support in ray ecosystem to carry out a text classification task using transfer learning. We will be referencing the example **[here](https://huggingface.co/docs/transformers/tasks/sequence_classification)**

The example carries out a text classification task on **[imdb dataset](https://huggingface.co/datasets/imdb)** and tries to classify the movie reviews as positive or negative. Huggingface library provides an easy way to build a model and the dataset to carry out this classification task. In this case we will be using **distilbert-base-uncased** model which is a **BERT** based model.

Huggingface has a **[built in support for ray ecosystem](https://docs.ray.io/en/releases-1.13.0/_modules/ray/ml/train/integrations/huggingface/huggingface_trainer.html)** which allows the huggingface trainer to scale on CodeFlare and can scale the training as we add additional gpus and can run distributed training across multiple GPUs that will help scale out the training.


### Getting all the requirements in place

In [2]:
! oc login --token=your-token --server=https://your-cluster

Logged into "https://your-cluster" as "kube:admin" using the token provided.

You have access to 74 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "rhods-notebooks".


In [3]:
! oc project default

Now using project "default" on server "https://your-cluster3".


Let's check that we have the necessary Hugging Face packages

In [18]:
import datasets
datasets.__version__

'2.6.1'

In [19]:
import transformers
transformers.__version__

'4.23.1'

In [20]:
import evaluate
evaluate.__version__

'0.3.0'

In [4]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration

In [5]:
# Create our cluster and submit appwrapper
cluster = Cluster(ClusterConfiguration(name='hfgputest', min_worker=1, max_worker=1, min_cpus=8, max_cpus=8, min_memory=16, max_memory=16, gpu=4, instascale=True, machine_types=["m5.xlarge", "p3.8xlarge"]))

Written to: hfgputest.yaml


In [6]:
cluster.up()

In [7]:
cluster.is_ready()

╭─────────────────────────╮
│   🚀 List of CodeFlare  │
│   clusters in queue🚀   │
│ +-----------+---------+ │
│ | Name      | Status  | │
│ +===========+=========+ │
│ | hfgputest | pending | │
│ |           |         | │
│ +-----------+---------+ │
╰─────────────────────────╯

(False, <CodeFlareClusterStatus.QUEUED: 2>)

In [12]:
cluster.status()

                  🚀 List of CodeFlare clusters 🚀                  
                                                                    
 ╭────────────────────────────────────────────────────────────────╮ 
 │   Owner                                                        │ 
 │   hfgputest                                        Active ✅   │ 
 │                                                                │ 
 │   URI: ray://hfgputest-head-svc.default.svc:10001              │ 
 │                                                                │ 
 │   ]8;id=572476;ray-dashboard-hfgputest-default.apps.prepfullinstall.psap.aws.rhperfscale.org\Dashboard🔗]8;;\                                                  │ 
 │                                                                │ 
 │                      Cluster Resources                         │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮     │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │     │ 
 │   │            │  │                                      │     │ 
 │   │  1    1    │  │  16G~16G     8           4           │     │ 
 │   │            │  │                                      │     │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯     │ 
 ╰────────────────────────────────────────────────────────────────╯

<RayClusterStatus.READY: 'ready'>

In [14]:
ray_cluster_uri = cluster.cluster_uri()

**NOTE**: Here we have created a custom cluster with a GPU. You can add more GPUs by changing the spec above. 

In [15]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for this training
runtime_env = {"pip": ["transformers", "datasets", "evaluate"]}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env)

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


**NOTE** : in this case since we are running a task for which we need additional pip packages. we can install those by passing them in the `runtime_env` variable

### Transfer learning code from huggingface

We are using the code based on the example **[here](https://huggingface.co/docs/transformers/tasks/sequence_classification)** . 

In [16]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.ml.train.integrations.huggingface import HuggingFaceTrainer

    dataset = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    #using a fraction of dataset but you can run with the full dataset
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

    print(f"len of train {small_train_dataset} and test {small_eval_dataset}")

    ray_train_ds = ray.data.from_huggingface(small_train_dataset)
    ray_evaluation_ds = ray.data.from_huggingface(small_eval_dataset)

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

        training_args = TrainingArguments("/tmp/hf_imdb/test", eval_steps=1, disable_tqdm=True, 
                                          num_train_epochs=1, skip_memory_metrics=True,
                                          learning_rate=2e-5,
                                          per_device_train_batch_size=16,
                                          per_device_eval_batch_size=16,                                
                                          weight_decay=0.01,)
        return transformers.Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

    scaling_config = {"num_workers": 4, "use_gpu": True} #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()


**NOTE:** This code will produce a lot of output and will run for **approximately 2 minutes.** As a part of execution it will download the `imdb` dataset, `distilbert-base-uncased` model and then will start transfer learning task for training the model with this dataset. 

In [17]:
#call the above cell as a remote ray function
ray.get(train_fn.remote())

(train_fn pid=249) Downloading and preparing dataset imdb/plain_text to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1...


100%|██████████| 3/3 [00:00<00:00, 710.50it/s]                                               


(train_fn pid=249) Dataset imdb downloaded and prepared to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 32.8kB/s]
Downloading: 100%|██████████| 483/483 [00:00<00:00, 559kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 4.77MB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 7.86MB/s]
 98%|█████████▊| 49/50 [00:31<00:00,  1.56ba/s]


(train_fn pid=249) len of train Dataset({
(train_fn pid=249)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=249)     num_rows: 100
(train_fn pid=249) }) and test Dataset({
(train_fn pid=249)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=249)     num_rows: 100
(train_fn pid=249) })


...

(BaseWorkerMixin pid=182, ip=10.128.64.16) 
(BaseWorkerMixin pid=182, ip=10.128.64.16) 
(BaseWorkerMixin pid=182, ip=10.128.64.16) Training completed. Do not forget to share your model on huggingface.co/models =)
(BaseWorkerMixin pid=182, ip=10.128.64.16) 
(BaseWorkerMixin pid=182, ip=10.128.64.16) 


(train_fn pid=249) == Status ==
(train_fn pid=249) Current time: 2022-11-04 04:27:16 (running for 00:02:18.35)
(train_fn pid=249) Memory usage on this node: 15.8/240.1 GiB
(train_fn pid=249) Using FIFO scheduling algorithm.
(train_fn pid=249) Resources requested: 5.0/10 CPUs, 4.0/4 GPUs, 0.0/22.35 GiB heap, 0.0/6.59 GiB objects
(train_fn pid=249) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2022-11-04_04-24-57
(train_fn pid=249) Number of trials: 1/1 (1 RUNNING)
(train_fn pid=249) +--------------------------------+----------+------------------+--------+------------------+-----------------+----------------------------+--------------------------+
(train_fn pid=249) | Trial name                     | status   | loc              |   iter |   total time (s) |   train_runtime |   train_samples_per_second |   train_steps_per_second |
(train_fn pid=249) |--------------------------------+----------+------------------+--------+------------------+-----------------+---------------------

(train_fn pid=249) 2022-11-04 04:27:16,323	WARNING util.py:214 -- The `process_trial_save` operation took 9.089 s, which may be a performance bottleneck.
(train_fn pid=249) 2022-11-04 04:27:16,323	WARNING trial_runner.py:856 -- Consider turning off forced head-worker trial checkpoint syncs by setting sync_on_checkpoint=False. Note that this may result in faulty trial restoration if a failure occurs while the checkpoint is being synced from the worker to the head node.


(train_fn pid=249) Result for HuggingFaceTrainer_50527_00000:
(train_fn pid=249)   _time_this_iter_s: 117.43207788467407
(train_fn pid=249)   _timestamp: 1667561227
(train_fn pid=249)   _training_iteration: 1
(train_fn pid=249)   date: 2022-11-04_04-27-07
(train_fn pid=249)   done: true
(train_fn pid=249)   epoch: 1.0
(train_fn pid=249)   experiment_id: 5c44a90f5c474864aee374a2ac1905e1
(train_fn pid=249)   experiment_tag: '0'
(train_fn pid=249)   hostname: hfgputest-worker-small-group-hfgputest-8f4mg
(train_fn pid=249)   iterations_since_restore: 1
(train_fn pid=249)   node_ip: 10.128.64.16
(train_fn pid=249)   pid: 146
(train_fn pid=249)   should_checkpoint: true
(train_fn pid=249)   step: 391
(train_fn pid=249)   time_since_restore: 124.04675316810608
(train_fn pid=249)   time_this_iter_s: 124.04675316810608
(train_fn pid=249)   time_total_s: 124.04675316810608
(train_fn pid=249)   timestamp: 1667561227
(train_fn pid=249)   timesteps_since_restore: 0
(train_fn pid=249)   train_loss: 

(train_fn pid=249) 2022-11-04 04:27:19,924	WARNING util.py:214 -- The `process_trial_save` operation took 2.028 s, which may be a performance bottleneck.
(train_fn pid=249) 2022-11-04 04:27:20,036	INFO tune.py:747 -- Total run time: 142.25 seconds (141.95 seconds for the tuning loop).


In [22]:
ray.shutdown()

In [23]:
cluster.down()

## Conclusion
As shown in the above example, you can easily run your Huggingface transfer learning tasks easily and natively on CodeFlare. You can scale them from 1 to n GPUs without requiring you to make any significant code changes and leveraging the native Huggingface trainer. 

Also refer to additional notebooks that showcase other use cases
In our next notebook [./02_codeflare_workflows_encoding.ipynb ] shows an sklearn example and how you can leverage workflows to run experiment pipelines and explore multiple pipelines in parallel on CodeFlare cluster. 
